In [27]:
import json
import os
import copy
file_path = '/Users/gongshukai/Desktop/ML RESEARCH/Ongoing Project/USPTO_LLM/Larrea/requests/result_uspto_requests.json'
file_path2 = '/Users/gongshukai/Desktop/ML RESEARCH/Ongoing Project/USPTO_LLM/Larrea/requests/uspto_requests.json'
output_path = '/Users/gongshukai/Desktop/ML RESEARCH/Ongoing Project/USPTO_LLM/Training_Info/'

In [50]:
with open(file_path, 'r') as raw_response: 
    lines = raw_response.readlines()
    total_data = [json.loads(line) for line in lines]

def extract_description(example_data):
    total_description = example_data[0]['messages'][1]['content']

    total_description = total_description.split('[Input]\nReaction ')[1:]

    for i, each_description in enumerate(total_description):
        first_split = each_description.index('\n')
        total_description[i] = each_description[first_split+1:]
    
    return total_description

example_data = total_data[29]

extract_description(example_data)

['19940329-US05298652-0037',
 '20090922-US07592360B2-0142',
 '20120628-US20120165313A1-0918']

In [75]:
def extract_gpt_response(example_data):
    total_response = example_data[1]['choices'][0]['message']['content']
    total_description = example_data[0]['messages'][1]['content'].split('[Input]\nReaction ')[1:]
    total_description_id=[each[:each.index(' description')] for each in total_description]

    total_response = total_response.split('Reaction ')[1:]

    reaction_id = []
    reaction_process = []
    for response in total_response:
        first_split = response.index('\n')
        last_split = response.index('P1')
        if response[:first_split] not in total_description_id or response[:first_split] in reaction_id:
            continue
        else:
            reaction_id.append(response[:first_split])
            reaction_process.append(response[first_split+1:last_split+2])
    
    return reaction_process, reaction_id

example_data = total_data[29]
extract_gpt_response(example_data)


(['R1.R2.R3>S1.E1.T1>M1\nM1>S2>E2>T2>P1',
  'R1.R2>S1.E1.T1>M1\nM1>S2>E2>T2>P1',
  'R1.R2>R3.S1.E1.T1>M1\nM1.R4>S2.E2.T2>M2\nM2>S3>E3>T1>P1'],
 ['19940329-US05298652-0037',
  '20090922-US07592360B2-0142',
  '20120628-US20120165313A1-0918'])

In [74]:
def check_invalid_answer(answer)->bool:
    # Check the validity of output heterogeneous graph
    # answer is a string in the form of ""R1.R2.R4>S1.E1.T1>M1\nM1.R3>E2.T2>P1""
    # split it by '\n' to get each 'substep'
    substeps = answer.split('\n')
    for substep in substeps:
        # If each step has more than 2 '>', it is invalid
        if substep.count('>') != 2:
            return False
        # If any entity with R and M is not in front of the first '>', it is invalid
        if 'R' in substep:
            if substep.index('R') > substep.index('>'):
                return False
        if 'M' in substep:
            if substep.index('M') > substep.index('>') and substep.index('M') < substep.rindex('>'):
                return False
        if 'P' in substep:
            if substep.index('P') < substep.rindex('>'):
                return False
        # If any entity with S, E, T, C is not behind the second '>', it is invalid
        if 'S' in substep:
            if substep.index('S') < substep.index('>') or substep.index('S') > substep.rindex('>'):
                return False
        if 'E' in substep:
            if substep.index('E') < substep.index('>') or substep.index('E') > substep.rindex('>'):
                return False
        if 'T' in substep:
            if substep.index('T') < substep.index('>') or substep.index('T') > substep.rindex('>'):
                return False
        if 'C' in substep:
            if substep.index('C') < substep.index('>') or substep.index('C') > substep.rindex('>'):
                return False
    return True

check_invalid_answer("[Input]\nReaction 20160908-US20160257671A2-1226 description:\nA mixture of (4aS,5S,8aS)-5-(3,5-dimethoxyphenoxymethyl)-1,1,4a,6-tetramethyl-1,2,3,4,4a,5,8,8a-octahydronaphthalene (5) (0.15 g, 0.45 mmol), Na2S (0.22 g, 2.7 mmol) and NMP (5.0 mL) were sealed in a microwave vessel and heated at 180\u00b0 C. with stirring for 1 h. The reaction mixture was cooled to room temperature, quenched with water and extracted with EtOAc. The organic layer was separated and concentrated to dryness. The residue was purified by flash column chromatography on silica gel (Hexanes/EtOAc, 6:1) to yield 3-{[(1 S,4aS,8aS)-2,5,5,8a-tetramethyl-1,4,4a,5,6,7,8,8a-octahydronaphthalen-1-yl]methoxy}-5-methoxyphenol (Compound No. 220) (0.058 g, 37% yield) as a pale brown oil. 1H NMR(CDCl3): \u03b46.05 (s, 1H), 6.00 (s, 2H), 5.50 (s, 1H), 4.60 (m, 1H), 4.05 (m, 1H), 3.85 (m, 1H), 3.75 (s, 3H), 2.15 (m, 1H), 2.00-1.80 (m, 3H), 1.68 (s, 3H), 1.61-0.95 (m, 6H), 0.88 (s, 3H), 0.84 (s, 6H); MS m/z 345 (C22H32O3+1) 196, obsd. 196.9.\n\nREACTANTs:\nR1: (4aS,5S,8aS)-5-(3,5-dimethoxyphenoxymethyl)-1,1,4a,6-tetramethyl-1,2,3,4,4a,5,8,8a-octahydronaphthalene,13;\nR2: Na2S,142;\n\n\nPRODUCTs:\nP1: 3-{[(1 S,4aS,8aS)-2,5,5,8a-tetramethyl-1,4,4a,5,6,7,8,8a-octahydronaphthalen-1-yl]methoxy}-5-methoxyphenol,522;\n\n\nSOLVENTs:\nS1: NMP,170;\n\n\nCATALYSTs:\nNone\n\nTIMEs:\nT1: 1 h,257;\n\n\nTEMPERATUREs:\nE1: 180\u00b0 C.,231;\nE2: room temperature,297;\n\n\nYIELD:\nY1: 37% yield,658;\n\n[Output]:\nReaction 20160908-US20160257671A2-1226\nR1.R2>S1.E1.T1>P1\n\n[Input]\nReaction 20160714-US20160200704A1-0536 description:\nTo a solution of 2-(2-chloro-phenyl)-ethylamine (10 g, 64 mmol, 1.0 eq) in DCM (150 mL) was added TEA (12.9 g, 128 mmol, 2.0 eq). The mixture was stirred at 0\u00b0 C., and methyl chloroformate (9.07 g, 96 mmol, 1.5 eq) was added under nitrogen atmosphere. The mixture was allowed to warm to rt and stirred for 1 h, concentrated and purified by silica gel chromatography (PE/EtOAc=5/1, v/v) to afford methyl 2-chlorophenethylcarbamate (9.0 g, 66%) as a colorless oil.\n\nREACTANTs:\nR1: 2-(2-chloro-phenyl)-ethylamine,17;\nR2: methyl chloroformate,168;\nR3: TEA,98;\n\n\nPRODUCTs:\nP1: methyl 2-chlorophenethylcarbamate,396;\n\n\nSOLVENTs:\nS1: DCM,75;\n\n\nCATALYSTs:\nNone\n\nTIMEs:\nT1: 1 h,306;\n\n\nTEMPERATUREs:\nE1: 0\u00b0 C.,157;\nE2: rt,287;\n\n\nYIELD:\nY1: 66%,438;\n\n[Output]:\nReaction 20160714-US20160200704A1-0536\nR1.R2.R3>S1.E1>E2.T1>P1")

False

In [88]:
rule1 = "(1) In each substep, there are at most two \">\", separating reactants, reaction conditions, and products! YOU MUST STRICTLY FOLLOW THIS FORMAT: Stuff before the first '>' is reactants \"Rx\"; Stuff in between \"> >\" is reaction conditions 'Sx', 'Cx', 'Ex', and 'Tx'; Stuff behind the second \">\" are products \"Px\", \"Mx\". "
rule2 = "(2) A standard format is \"Rx.Ry > Sx.Cx.Ex.Tx > Mx\". If there is no reaction condition, don't fill anything in between \"> >\". DO NOT include Yields (Y1) into the reaction substeps because the reaction substeps always end with a \"P1\". "
clarification = "Give me heterogeneous graphs only with no introduction or explanation."

def modified_reaction_string(reaction_id, total_description, response, mode=0):
    if mode == 0:   # Valid reaction, write into uspto_multiple_step.json
        # If "response" has a substring of "Don't make the mistake of", find the start of the substring
        pattern = "(Do not make the mistake of"
        start = total_description.find(pattern)
        if start != -1:
            total_description = total_description[:start]
        return total_description
    elif mode == 1: # Invalid reaction, Write into uspto_invalid.json
        pattern = "(Do not make the mistake of"
        start = total_description.find(pattern)
        if start != -1:
            total_description = total_description[:start]
        return "[Input]\nReaction " + reaction_id + " description:\n" + total_description + "(Do not make the mistake of '" + response + "'. Always remember "+rule1+rule2+clarification+")"

test_string = "To dichloromethane (2 ml), 5-[1,1-dimethoxy-2-(4-methoxybenzyloxy)ethyl]-2-(2,4-dimethylphenyl)-4-methylthiazole (35 mg) prepared in the Step 31-1-3 was added, and trifluoroacetic acid (0.3 ml) and water (0.1 ml) were added thereto. Thereafter, the mixture was stirred at a room temperature for 5 hours. To the reaction solution, a saturated sodium bicarbonate solution was added, and then the resulting mixture was subjected to extraction with chloroform. The extract was washed with a saturated saline solution, and then dried over sodium sulfate and concentrated under a reduced pressure. The residue was purified by thin-layer chromatography (hexane:ethyl acetate=3:1, toluene:ethyl acetate=3:1) to give the title compound (18 mg, 57%).\n\nREACTANTs:\nR1: 5-[1,1-dimethoxy-2-(4-methoxybenzyloxy)ethyl]-2-(2,4-dimethylphenyl)-4-methylthiazole,27;\nR2: trifluoroacetic acid,164;\nR3: sodium bicarbonate,342;\nR4: dichloromethane,3;\n\n\nPRODUCTs:\nP1: title compound,712;\n\n\nSOLVENTs:\nS1: water,198;\n\n\nCATALYSTs:\nNone\n\nTIMEs:\nT1: 5 hours,295;\n\n\nTEMPERATUREs:\nE1: room temperature,274;\n\n\nYIELD:\nY1: 57%,735;\n\n[Output]:\n(Do not make the mistake of 'R1.R2.R4>S1.E1.T1>M1\nM1.R3>E2>T2>P1'. Always remember (1) In each substep, there are at most two \">\", separating reactants, reaction conditions, and products! YOU MUST STRICTLY FOLLOW THIS FORMAT: Stuff before the first '>' is reactants \"Rx\"; Stuff in between \"> >\" is reaction conditions 'Sx', 'Cx', 'Ex', and 'Tx'; Stuff behind the second \">\" are products \"Px\", \"Mx\". (2) A standard format is \"Rx.Ry > Sx.Cx.Ex.Tx > Mx\". If there is no reaction condition, don't fill anything in between \"> >\". DO NOT include Yields (Y1) into the reaction substeps because the reaction substeps always end with a \"P1\". Give me heterogeneous graphs only with no introduction or explanation.)"

modified_reaction_string("123",test_string, "a>b>c", mode=0)

'To dichloromethane (2 ml), 5-[1,1-dimethoxy-2-(4-methoxybenzyloxy)ethyl]-2-(2,4-dimethylphenyl)-4-methylthiazole (35 mg) prepared in the Step 31-1-3 was added, and trifluoroacetic acid (0.3 ml) and water (0.1 ml) were added thereto. Thereafter, the mixture was stirred at a room temperature for 5 hours. To the reaction solution, a saturated sodium bicarbonate solution was added, and then the resulting mixture was subjected to extraction with chloroform. The extract was washed with a saturated saline solution, and then dried over sodium sulfate and concentrated under a reduced pressure. The residue was purified by thin-layer chromatography (hexane:ethyl acetate=3:1, toluene:ethyl acetate=3:1) to give the title compound (18 mg, 57%).\n\nREACTANTs:\nR1: 5-[1,1-dimethoxy-2-(4-methoxybenzyloxy)ethyl]-2-(2,4-dimethylphenyl)-4-methylthiazole,27;\nR2: trifluoroacetic acid,164;\nR3: sodium bicarbonate,342;\nR4: dichloromethane,3;\n\n\nPRODUCTs:\nP1: title compound,712;\n\n\nSOLVENTs:\nS1: water

In [77]:
def output_structured_data(request_in_each_line = 0, threshold = 2):
    # Read raw response first
    with open(file_path, 'r') as raw_response: 
        lines = raw_response.readlines()
        total_data = [json.loads(line) for line in lines]
    
    modified_request = copy.deepcopy(total_data[0][0])     # Fixed Prompt
    modified_request['messages'][1]['content'] = ""

    for dataline in total_data:
        total_description = extract_description(dataline)
        total_response, reaction_id = extract_gpt_response(dataline)

        for idx, response in enumerate(total_response):
            if check_invalid_answer(response):
                with open(output_path + "uspto_multiple_step.json", 'a') as structured_data:
                    structured_data.write(json.dumps({'reaction_id': reaction_id[idx], 'description': modified_reaction_string(reaction_id[idx], total_description[idx], response, mode=0), 'response': response}) + '\n')
            else:
                with open(output_path + "uspto_invalid.json", 'a') as invalid_data:
                    if request_in_each_line < threshold:
                        modified_request['messages'][1]['content'] += modified_reaction_string(reaction_id[idx], total_description[idx], response, mode=1)
                        request_in_each_line += 1
                    else:
                        request_in_each_line = 0
                        invalid_data.write(json.dumps(modified_request) + '\n')
                        modified_request['messages'][1]['content'] = ""
                        modified_request['messages'][1]['content'] += modified_reaction_string(reaction_id[idx], total_description[idx], response, mode=1)
                        request_in_each_line += 1
        
    # Less than threshold requests in one line, still need to write them
    if request_in_each_line != 0:
        with open(output_path + "uspto_invalid.json", 'a') as invalid_data:
            invalid_data.write(json.dumps(modified_request) + '\n')
    # If no error arise, print success
    print("Structured data output success!")


In [34]:
def write_over_original_file(
        read_file = output_path + "uspto_invalid.json",
        write_path = file_path2
        ):
    with open(read_file, 'r') as invalid_file:
        lines = invalid_file.readlines()
        with open(write_path, 'w') as raw_file:
            for line in lines:
                raw_file.write(line)


In [78]:
if os.path.exists(output_path + "uspto_multiple_step.json"):
    os.remove(output_path + "uspto_multiple_step.json")
if os.path.exists(output_path + "uspto_invalid.json"):
    os.remove(output_path + "uspto_invalid.json")
output_structured_data()

Structured data output success!


In [90]:
if os.path.exists(file_path):
    print("result_uspto_request.json removed!")
    os.remove(file_path)
    print("Writing uspto_invalid.json over uspto_request.json for API-recalling")
    write_over_original_file()
else:
    print("uspto_request.json does not exist!")

# API calling on the new uspto_request.json
# Getting the new result_uspto_request.json

result_uspto_request.json removed!
Writing uspto_invalid.json over uspto_request.json for API-recalling


In [91]:
if os.path.exists(output_path + "uspto_invalid.json"):
    os.remove(output_path + "uspto_invalid.json")
output_structured_data()

Structured data output success!
